# Earthquake risk in Greece

This project is quite different from a software development task.

Don't hesitate to contact us, if you have any doubts on what is asked or if you encounter error will using the notebook.

In [ ]:
%reload_ext autoreload
%autoreload 2

## Constraints

+ 🚨 Only cells with the comment `# NOTE: Fill me!` should be filled
+ 🚨 Notebook should be saved and commited **with** outputs for the submission


+ ⚠️ The solution only requires packages listed in the `requirements/requirements.txt`
+ ⚠️ Unit tests should be favored when asked to write tests 
+ ⚠️ Tests must automatically be detected running `pytest`
+ ⚠️ Requested method signature should be inferred from this notebook


## Note

+ The `assert` statements in the notebook are here to guide the project.
However, successful `assert` statements does not guaranty that your code is correct.

## Setup

In a Python >= 3.8 virtual env, run:

In [ ]:
! pip install -r ../requirements/requirements.txt
! pip install --no-deps -e ..

## Tests

In [ ]:
! cd .. && pytest

---

# Context

A client asks for an insurance of their asset, located at `(35.025, 25.763)` in Greece.

The client wishes to receive a payout under the following conditions:

+ earthquake of magnitude `4.5` or higher within `10km`: full payout
+ earthquake of magnitude `5.5` or higher within `50km`: `75%` payout
+ earthquake of magnitude `6.5` or higher within `200km`: `50%` payout

In the event of aftershocks, a payout can only occur once a year using the maximal value.

## Example

If in the same year:

* an earthquake of magnitude `6.8` occurs within `200km`
* **and** an aftershock of magnitude `5.8` occurs within `50km`

the client receives a `75%` payout, and not a `125%` payout.

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime

from earthquakes.tools import (
    DISTANCE_COLUMN,
    LATITUDE_COLUMN,
    LONGITUDE_COLUMN,
) 

# Earthquake data

The US Geological Service (USGS) provides CSV data through their [API](https://earthquake.usgs.gov/fdsnws/event/1/).

Use it to retrieve earthquake information.

In the module `earthquakes.usgs_api`:
+ Implement the function `get_earthquake_data`,
+ The function will retrieve the earthquake data of the area of interest for the past 200 years,
+ The implementation must use the `urllib` python package,
+ The API request url must be build in a dedicated function `build_api_url`,
+ Tests should be provided for `build_api_url`.

Note: Earthquakes after the 21-10-2021 should not be considered.

In [ ]:
from earthquakes.usgs_api import get_earthquake_data

In [ ]:
# NOTE: Fill me!

latitude = "fill with proper object"
longitude = "fill with proper object"
radius = "fill with proper object"
minimum_magnitude = "fill with proper object"

In [ ]:
# NOTE: This request may take significant time (>10s)
earthquake_data = get_earthquake_data(
    latitude=latitude,
    longitude=longitude,
    radius=radius,
    minimum_magnitude=minimum_magnitude,
    end_date=datetime(year=2021, month=10, day=21)
)

In [ ]:
assert isinstance(earthquake_data, pd.DataFrame)
assert len(earthquake_data) == 656

## Warning

The next test may fail because USGS regularly updates their earthquake database.

The dataframe obtained should over be similar to that presented bellow.

Please contact us if there is an error.

In [ ]:
expected_earthquake_data = pd.DataFrame([
        ["2021-10-12T09:24:05.099Z", 35.1691, 26.2152, 20.0, 6.4, "mww", np.nan, 19.0, 0.860, 0.46,"us", "us6000ftxu", "2021-12-18T19:58:57.040Z", "4 km SW of Palekastro, Greece", "earthquake", 6.1, 1.8, 0.048, 42.0, "reviewed", "us", "us"],
        ["2021-10-03T14:31:27.622Z", 35.1442, 25.2375, 10.0, 4.6, "mb", np.nan, 119.0, 0.318, 0.64, "us", "us6000fsp1", "2021-12-10T21:14:19.040Z", "2 km W of Arkalochóri, Greece", "earthquake", 5.0, 1.9, 0.165, 13.0, "reviewed", "us", "us"],
        ["2021-09-29T11:54:48.885Z", 35.0268, 25.1561, 10.0, 4.6, "mb", np.nan, 69.0, 0.339, 0.83, "us", "us6000fq3y", "2021-12-04T14:27:58.040Z", "2 km N of Pýrgos, Greece", "earthquake", 5.1, 1.3, 0.068, 64.0, "reviewed", "us", "us"],
        ["2021-09-28T15:13:16.867Z", 35.2054, 25.2791, 10.0, 4.7, "mb", np.nan, 58.0, 0.329, 0.70, "us", "us7000ff84", "2021-12-04T14:30:09.040Z", "1 km N of Thrapsanón, Greece", "earthquake", 6.9, 1.8, 0.067, 73.0, "reviewed", "us", "us"],
        ["2021-09-28T04:48:08.650Z", 35.0817, 25.2018, 10.0, 5.3, "mww", np.nan, 43.0, 0.328, 0.94, "us", "us7000ff36", "2021-12-04T14:30:04.040Z", "9 km SW of Arkalochóri, Greece", "earthquake", 4.5, 1.7, 0.046, 45.0, "reviewed", "us", "us"],
    ],
    columns=['time', 'latitude', 'longitude', 'depth', 'mag', 'magType', 'nst', 'gap', 'dmin', 'rms', 'net', 'id', 'updated', 'place', 'type', 'horizontalError', 'depthError', 'magError', 'magNst', 'status', 'locationSource', 'magSource']
)

In [ ]:
earthquake_data_sample = earthquake_data[
    earthquake_data["time"].isin(expected_earthquake_data["time"])
]

pd.testing.assert_frame_equal(earthquake_data_sample, expected_earthquake_data)

# Distance

We wish to compute the the historical payouts (i.e. the payouts that would have occurred for the past 200 years).

To compute the historical payouts, we need to know the distance between each earthquake and our client's asset.

The distance between two points on a sphere is the [Haversine distance](https://en.wikipedia.org/wiki/Haversine_formula). In the module `eathquakes.tools`:
- Implement and test the function `get_haversine_distance`,
- Use `earthquakes.tools.EARTH_RADIUS` (6378km) as an approximation of the radius of Earth.

In [ ]:
from earthquakes.tools import get_haversine_distance

distances = get_haversine_distance(earthquake_data[LATITUDE_COLUMN], earthquake_data[LONGITUDE_COLUMN], latitude, longitude)

earthquake_data[DISTANCE_COLUMN] = distances

## Historical payouts and burning costs

### Payout

The historical payouts are a map `year -> payout in %`.

eg: `1950: 50` for a payout of `50%` in 1950.

Payouts are NOT given per event, but per year.

This map can take the form of a python `dict` or of a pandas `Series`. 

### Burning cost

The `burning cost` is the average of payouts over a time range.

In this project, the burning cost should be expressed in `%`. 

### Payout structure

The payout structure is:

| Radius | Magnitude | Payout |
|--------|-----------|--------|
| 10km   | 4.5       | 100 %  |
| 50km   | 5.5       |  75 %  |
| 200km  | 6.5       |  50 %  |

Payouts can occur only once in a given year.

In the module `earthquakes.tools`:
+ Implement the functions `compute_payouts` and `compute_burning_cost`,
+ Tests for these functions are not required.

### Example

A payout `{1950: 50, 1992: 75}` means that we would have paid our client
+ in 1950, for `50%` of the insured amount (called 'limit')
+ in 1992, for `75%` of the limit

The burning cost over the `1922-2021` period would be `1.25%`.

The burning cost over the `1972-2021` period would be `1.5%`.

In [ ]:
from earthquakes.tools import compute_payouts, compute_burning_cost

In [ ]:
# NOTE: Fill me!

payout_structure = "fill with proper object"

In [ ]:
payouts = compute_payouts(earthquake_data, payout_structure)

In [ ]:
# Depending on the return type of compute_payouts, the following line needs to be adjusted.
# - pd.Series:
payout_values = np.array(payouts.values)
# - dict:
# payout_values = np.array(list(payouts.values()))
assert np.max(payout_values) > 1
assert np.max(payout_values) <= 100

In [ ]:
burning_cost = compute_burning_cost(payouts, start_year=1952, end_year=2021)

In [ ]:
np.testing.assert_allclose(burning_cost, 10.71, atol=1e-2)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
years = range(1921, 2021)
plt.plot(
    years, 
    [
        compute_burning_cost(payouts, start_year=start_year, end_year=2021) 
        for start_year in years
    ]
)
plt.gca().invert_xaxis()

# Large asset portfolio - async requests

Our client also whishes to cover a large amount of properties all over Europe.

In order to speed-up the requests to the USGS API, in the module `earthquakes.usgs_api`:
- Implement the `async` function `get_earthquake_data_for_multiple_locations`,
- The implementation should use the `asyncio` and `aiohttp` libraries,
- The solution should re-use some of the functions already written,
- Tests are not required for any of the functions.

Note: it is possible that the notebook autoreload feature doesn't work for `async` functions - a kernel restart may be necessary after each modifications.

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime

from earthquakes.tools import (
    LATITUDE_COLUMN,
    LONGITUDE_COLUMN,
)

In [ ]:
number_of_assets = 10
# NOTE: limiting to number of assets so that the query doesn't take too long.

In [ ]:
random_state = np.random.RandomState(0)

random_values = random_state.random(2*number_of_assets)

latitudes = random_values[::2] * 20 + 35.0
longitudes = random_values[1::2] * 25 + 3.0

In [ ]:
from earthquakes.usgs_api import get_earthquake_data_for_multiple_locations

In [ ]:
# NOTE: Fill me!
assets = "fill with proper object"

In [ ]:
# NOTE: This request may take significant time (>10s)
earthquake_data = await get_earthquake_data_for_multiple_locations(  # type: ignore
    assets, 
    radius=200, 
    minimum_magnitude=4.5, 
    end_date=datetime(year=2021, month=10, day=21)
)

In [ ]:
assert isinstance(earthquake_data, pd.DataFrame)
assert len(earthquake_data) == 1036